In [1]:
import numpy as np
import pandas as pd
import sklearn as sk

train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

train_set.drop(['ID_code'], axis = 1, inplace = True)
train_set.head()

idx = train_set.columns.values[1:]
for df in [test_set, train_set]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)

from sklearn.model_selection import train_test_split
y = np.array(train_set)[:, 0]
X = np.array(train_set)[:, 1:]

from sklearn.preprocessing import StandardScaler
scaller = StandardScaler()
X = scaller.fit_transform(X)

from sklearn.preprocessing import QuantileTransformer
q_scaller = QuantileTransformer()
X = q_scaller.fit_transform(X)

np.shape(X)

## Random Forest

In [4]:
# from sklearn.ensemble import RandomForestClassifier

In [5]:
# classifier = RandomForestClassifier(class_weight='balanced', verbose=True)
# classifier.fit(X_train, y_train)

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
# classifier.fit(X, y)

## Naive

In [8]:
from sklearn.naive_bayes import GaussianNB
classifier_2 = GaussianNB()

classifier_2.fit(X,y)

GaussianNB(priors=None, var_smoothing=1e-09)

## ANN

In [9]:
import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape = (208,)))
model.add(Dropout(rate = 0.4))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu', input_shape = (128,)))
model.add(Dropout(rate = 0.4))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu', input_shape = (64,)))
model.add(Dropout(rate = 0.4))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu', input_shape = (32,)))
model.add(Dropout(rate = 0.4))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid', input_shape = (8,)))
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               26752     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
__________

In [11]:
# model.fit(X, y, batch_size = 200, epochs = 100, class_weight={0:0.6, 1: 1})

## XGB

## Wyniki

In [27]:
test_X = np.array(test_set)[:, 1:]
test_X = test_X.astype(float)
test_X = scaller.transform(test_X)
test_X = q_scaller.transform(test_X)

In [28]:
model.load_weights('nn_weights')

In [29]:
y_pred = model.predict(test_X)


In [30]:
y_pred_NB = classifier_2.predict_proba(test_X)
y_pred_NB = y_pred_NB[:,1]
y_pred_NB = y_pred_NB.reshape([200000,1])
y_pred_NB.shape

(200000, 1)

In [31]:
y_pred = y_pred.astype(float)
y_pred_mean = y_pred/2 + y_pred_NB/2

In [32]:
y_pred_mean

array([[0.24934249],
       [0.25597436],
       [0.14121109],
       ...,
       [0.0342946 ],
       [0.08446128],
       [0.20905647]])

In [ ]:
# model.save_weights('nn_weights')

In [34]:
anwser = pd.concat([test_set['ID_code'],
                     pd.DataFrame(y_pred_mean)], axis=1)


anwser.set_axis(['ID_code', 'target'], axis = 1)
anwser.head()

anwser.to_csv('anwser.csv', index=False)

/home/hakunamatata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  """


,ID_code,target
0,test_0,0.249342
1,test_1,0.255974
2,test_2,0.141211
3,test_3,0.283435
4,test_4,0.191467


In [36]:
anwser['target'].mean()

0.1440862645444208